<h1>Data Prep for Naive Bayes</h1>

<hr style="height:.9px;border:none;color:#333;background-color:#333;" />

To start our assignment, we imported the csv file and created a list to specify the headers of each column to be used in the "name" argument during the reading of the csv file, we gathered basic information of our data, and searched for missing values by using the .info() and .isnull().sum() methods. 
<br><br>
The .info() function helped us check the information of our data, like validation of data types and number of columns we had on the dataset. 
The .isnull() function we used, at first showed that everything was false, meaning that we supposedly has no missing values on our data. However, we suspected that some missing values could have been codified in a different way and not as NaN or the default missing values that the read_csv method could identify. For this, we decided to create a loop to perform value_counts on each of our columns and print these values in a sorted way. By doing this, we found out that there were some question marks in the data that could have been used to mark missing values.
<br><br>
Further, by looking at the information we have gathered, we identified that a space was located before of every question mark representing the missing values, so we added the skipinitialspace argument and set it to True to make sure that we cleared the space before the question marks. Then, we added our "?" in the na_values argument among with the default values so that these could be identified as missing values in our data set.


In [ ]:
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns
import statsmodels.formula.api as smf # regression modeling
from sklearn.model_selection import train_test_split # train/test split


# specifying a file 
file = "./US Census Above 50k Predictor.csv"

#creating list of column names
header_names = ["Age",
                "Job/occupation",
                "Type of employment",
                "Current hours of work/week",
                "Capital gain/loss",
                "Highest level of education",
                "Completed years of education",
                "Marital status",
                "Relationship inside the household",
                "Ethnicity",
                "Gender",
                "Country of birth", 
                "Income"]

# reading file into Python through pandas
# using names to asign a string as a column name
us_census = pd.read_csv(filepath_or_buffer  = file,
                       sep = ',',
                       names = header_names,
                        #using skipininitialspace after finding out with a loop there were '?' in the values
                       skipinitialspace = True,
                        #converting the '?' to a NaN value
                       na_values = ['','#N/A', '#N/A N/A', '#NA', '-1.#IND', '-1.#QNAN', '-NaN', '-nan',
                                    '1.#IND', '1.#QNAN', '<NA>', 'N/A', 'NA', 'NULL', 'NaN', 'n/a',
                                    'nan', 'null','?'])

<br><br>
<strong>Validation of identification of missing values</strong>
<br>
We checked that what we did to identify the missing values was working by using the isnull().sum() functions. We could see that now we have missing values for Job/occupation, Type of employment, and Country of birth.
<br>
By running the value_counts() function on these variables, we identified how many values we had on each of their categories and we were able to compare these with the amount of missing values. 
<br><br>

In [ ]:
# looking for NaN values
us_census.isnull().sum(axis = 0)

In [ ]:
us_census['Job/occupation'].value_counts(normalize = False,
                                         sort      = False,
                                         ascending = False,
                                         dropna    = False).sort_index()

In [ ]:
us_census['Type of employment'].value_counts(normalize = False,
                                             sort      = False,
                                             ascending = False,
                                             dropna    = False).sort_index()

In [ ]:
us_census['Country of birth'].value_counts(normalize = False,
                                           sort      = False,
                                           ascending = False,
                                           dropna    = False).sort_index().round(decimals = 2)

<br><br>
<strong>Flagging the missing values</strong>
<br>
To point out the missing values we created a loop to flag them and added new columns at the end of our dataset with "1" were the missing values were found. These new columns have the same name as the original column were the missing values were found, but a letter "m" from missing was added at the beginning.
<br><br>

In [ ]:
# flag the missing values
for column in us_census:

    if us_census[column].isnull().astype(int).sum() > 0:
        us_census['m_' + column] = us_census[column].isnull().astype(int)

<br><br>
<strong>Describing categorical variables</strong>
<br>
To describe our categorical variables we used the function .describe() to help us know the number of categories on each categorical variable, how many values we could find, and the mode for each categorical variable. The argument include was used to obtain only this type of variables.
<br><br>
We found out that some of our variables wouldn't be the best approach to use as a dummy variables if we didn't group them into smaller categories, because they have too many categories. For example, Country of birth had 41 possible values.
<br><br>
We also used the value_counts() method with a for loop to identify the frequency of each categorical value, because later on we would need this as an input to create smaller categories among our variables. It is important to mention that this cell of code was commented out so that it is not run unless someone needs to look at the frequencies again.
<br><br>

In [ ]:
# describing categorical variables
us_census.describe(include = 'object')

In [ ]:
# # finding frequency for each categorical values

# for col in us_census.select_dtypes(include = 'object'):
#     count_values_columns = us_census[col].value_counts(normalize = True,
#                                                        sort      = True,
#                                                        ascending = True).round(decimals = 2)
#     print(count_values_columns)
#     print('-'*90)


<br><br>
<strong>Creating a separate DataFrame for our target variable</strong>
<br>
To be sure that we have our target variable in a separate DataFrame for future reference we decided to separate it and name it "y".
<br><br>

In [ ]:
# creating a new DataFrame for Income, our target variable
y= pd.DataFrame(data = us_census, columns = ['Income'])

<br><br>
<strong>Treatment of missing values</strong>
<br>
To treat the missing values we first created a new DataFrame were we dropped the missing values. This was done so that we could created a variable to identify the mode of this data set and to compare our new data once we imputed the missing values.
<br><br>
By using value_counts on our columns that had missing values, and sorting them to have the most repeated value or mode at the top, and then calling for the index 0, we were able to identify the mode. This value was saved into a new variable that would be used in the imputation process.
<br><br>
However, for the count of the values of Job/occupation, we decided to use the second most repeated occupation by calling the index 1. We did this because we found out that the mode was 'Prof-specialty' and this value was going to be imputed to observations which actually wouldn't have enough years of education to be lawfully allowed to work as a professional specialty. Moreover, the second most repeated value of 'Exec-managerial' is not restricted by law regarding the number of years of education required.
<br>
For our other missing values of Type of employment and Country of birth, the mode did make sense so we didn't need to modify our index.
<br><br>
Further, by using .fillna(), we were able to impute the missing values with the mode or second most repeated value. Also, we validated each imputation to make sure which value was used for the imputation.
<br><br>

In [ ]:
# Dropping the missing values and creating a new DataFrame
census_dropped = pd.DataFrame.copy(us_census)

census_dropped = census_dropped.dropna()

In [ ]:
# missing values for Job/occupation
job_mode = us_census['Job/occupation'].value_counts(normalize = False,
                                    sort      = True,
                                    ascending = False).index[1]

# fill NA with mode
us_census['Job/occupation'].fillna(value = str(job_mode), #value for filling the NA
                                   inplace = True) #replacing the Na

In [ ]:
# missing values for Type of employment
employment_mode = us_census['Type of employment'].value_counts(normalize = False,
                                    sort      = True,
                                    ascending = False).index[0]

# fill NA with mode
us_census['Type of employment'].fillna(value = str(employment_mode), #value for filling the NA
                                   inplace = True) #replacing the Na



In [ ]:
# missing values for Country of birth
country_mode = us_census['Country of birth'].value_counts(normalize = False,
                                    sort      = True,
                                    ascending = False).index[0]

# fill NA with mode
us_census['Country of birth'].fillna(value = str(country_mode), #value for filling the NA
                                   inplace = True) #replacing the Na



In [ ]:
# validating the missing values were filled with the second most repeated value
us_census.loc[ : , ['m_Job/occupation','Job/occupation']] [us_census.loc[ : , 'm_Job/occupation'] == 1]

In [ ]:
# validating the missing values are filled with the mode
us_census.loc[ : , ['m_Type of employment','Type of employment']] [us_census.loc[ : , 'm_Type of employment'] == 1]

In [ ]:
# validating the missing values are filled with the mode
us_census.loc[ : , ['m_Country of birth','Country of birth']] [us_census.loc[ : , 'm_Country of birth'] == 1]

In [ ]:
# validation to make sure that we don't have missing values in our imputed data
print(census_dropped.isnull().sum())

<br><br>
<strong>Validation of the treatment applied to missing values</strong>
<br>
To compare our original data with the imputed data we created 3 histograms including both distributions in the same graph.
<br><br>

<em><strong>Distribution of Job/occupation</strong></em>
<br>
By looking at the graph, we found out that for the Job/occupation we did create a spike in the Exec-managerial frequency when there actually wasn't a big difference in the original data. However, we're satisfied with this imputation because as mentioned before, this occupation doesn't necessarily require a certain number of years of education, and so this could be the way to achieve the most minor disruption from the original data.
<br><br>

In [ ]:
# overlay the original and imputed distributions for Job/occupation
fig, ax = plt.subplots(figsize = [20, 10],
                      sharex = True, #sharing x axis between visualization
                      sharey = True) #sharing y axis between visualization

# histogram for Job/occupation - oringal data
sns.histplot(data  = census_dropped,
             x     = 'Job/occupation',
             bins  = 'fd',
             kde   = True, # drawing theoretical distribution
             color = 'red')


# histogram for Job/occupation - imputed data
sns.histplot(data  = us_census,
             x     = 'Job/occupation',
             bins  = 'fd',
             kde   = True, # drawing theoretical distribution
             color = 'black')


# titles, labels, and formatting
plt.title(label   = "Distribution of Job/occupation")
plt.xlabel(xlabel = 'Job/occupation')
plt.ylabel(ylabel = 'Frequency')
plt.xlim(-0.5, 14) # setting x-axis range
plt.ylim(0.0, 5000) # setting y-axis range


# adds legend
plt.legend(labels =  ['original distribution',
                      'imputed distribution'])

# compile and display plot
plt.tight_layout()
plt.show()

<em><strong>Distribution of Type of employment</strong></em>
<br>
For the type of employment, by looking at our graph we can see the private jobs were increased by our imputation. However, we already had a big difference in the frequency of this type of employment in the original data when compared to the others. So, we take this imputation the best to reduce the interference of the original information. 
<br><br>

In [ ]:
# overlay the original and imputed distributions for Type of employment
fig, ax = plt.subplots(figsize = [20, 10],
                      sharex = True, #sharing x axis between visualization
                      sharey = True) #sharing y axis between visualization

# histogram for Type of employment - oringal data
sns.histplot(data  = census_dropped,
             x     = 'Type of employment',
             bins  = 'fd',
             kde   = True, # drawing theoretical distribution
             color = 'red')

# histogram for Type of employment - imputed data
sns.histplot(data  = us_census,
             x     = 'Type of employment',
             bins  = 'fd',
             kde   = True, # drawing theoretical distribution
             color = 'black')


# titles, labels, and formatting
plt.title(label   = "Distribution of type of employment")
plt.xlabel(xlabel = 'Type of employment')
plt.ylabel(ylabel = 'Frequency')
plt.xlim(-0.5, 7.5) # setting x-axis range
plt.ylim(0.0, 20000) # setting y-axis range


# adds legend
plt.legend(labels =  ['original distribution',
                      'imputed distribution'])

# compile and display the plot
plt.tight_layout()
plt.show()

<em><strong>Distribution of Country of birth</strong></em>
<br>
For the country of birth, the histogram shows that the imputation of the United States as the country of birth increases the difference among the number of observations from this country versus the others. However, this increase makes sense because the data came from the census of the US population so we would expect this country with a vast majority of observations.
<br><br>

In [ ]:
# overlay the original and imputed distributions for Country of birth
fig, ax = plt.subplots(figsize = [20, 10],
                      sharex = True, #sharing x axis between visualization
                      sharey = True) #sharing y axis between visualization

# histogram for Country of birth - oringal data
sns.histplot(data  = census_dropped,
             x     = 'Country of birth',
             bins  = 'fd',
             kde   = True, # drawing theoretical distribution
             color = 'red')

# histogram for Country of birth - imputed data
sns.histplot(data  = us_census,
             x     = 'Country of birth',
             bins  = 'fd',
             kde   = True, # drawing theoretical distribution
             color = 'black')


# titles, labels, and formatting
plt.title(label   = "Distribution of Country of birth")
plt.xlabel(xlabel = 'Country of birth')
plt.ylabel(ylabel = 'Frequency')
plt.xlim(0.0, 20) # setting x-axis range
plt.ylim(0.0, 25000) # setting y-axis range


# adds legend
plt.legend(labels =  ['original distribution',
                      'imputed distribution'])

# compile and display the plot
plt.tight_layout()
plt.show()

<br><br>
<strong>Categorizing variables</strong>
<br>
As mentioned before, since all of our variables had too many categories or values, and getting dummy variables for all of them will significantly reduce our degrees of freedom, and create stratas with not enough observations, we decided to group them based on their similar characteristics.
<br>

<em><strong>Grouping non categorical variables</strong></em>
<br>
First used a loop to group all of our non categorical variables into predetermined bins, and create a new column where the resulting bins will be identified regarding the original data.
<br>
<em>-Age: </em>Grouped by using descriptive statistics and quartiles.
<br>
<em>-Current hours of work/week: </em>Grouped by looking at job descriptions in the US.
<br>
<em>-Capital gain/loss: </em>Grouped by creating groups of negative vs positive, and dividing the positive in groups of 5K.
<br>
<em>-Completed years of education: </em>Grouped by education levels for the US.
<br>

In [ ]:
# creating categories for non categorical variables 

#copying into a new dataframe so that the original imputated data is not modified
us_census2 = us_census.copy()

# variable Age
us_census2 ['Cat_Age'] = 0

# for loop with iterrows()
for index, col in us_census2.iterrows():
    
    
    # conditionals to change the values in the new column, defined by looking at descriptive statistics
    if us_census2.loc[index, 'Age'] < 28:
        us_census2.loc[index, 'Cat_Age'] = '[0 - 28)'
        
        
    elif us_census2.loc[index, 'Age'] < 37:
        us_census2.loc[index, 'Cat_Age'] = '[28 - 37)'
        
        
    elif us_census2.loc[index, 'Age'] < 48:
        us_census2.loc[index, 'Cat_Age'] = '[37 - 48)'
        
        
    elif us_census2.loc[index, 'Age'] <= 90:
        us_census2.loc[index, 'Cat_Age'] = '[48 - 90)'
        
        
    elif us_census2.loc[index, 'Age'] >= 91:
        us_census2.loc[index, 'Cat_Age'] = '[91 - inf)'
    
    
    # safety net
    else:
        us_census2.loc[index, 'Cat_Age'] = 'Age not defined'
        

# variable Current hours of work/week
us_census2 ['Cat_Current hours of work/week'] = 0

# for loop with iterrows()
for index, col in us_census2.iterrows():
    
    
    # conditionals to change the values in the new column, defined by looking at job descriptions for the US
    if us_census2.loc[index, 'Current hours of work/week'] < 20:
        us_census2.loc[index, 'Cat_Current hours of work/week'] = 'Half time [0 - 20)'
        
        
    elif us_census2.loc[index, 'Current hours of work/week'] < 30:
        us_census2.loc[index, 'Cat_Current hours of work/week'] = 'Part time [20 - 30)'
        
        
    elif us_census2.loc[index, 'Current hours of work/week'] < 41:
        us_census2.loc[index, 'Cat_Current hours of work/week'] = 'Full time [30 - 41)'
        
        
    elif us_census2.loc[index, 'Current hours of work/week'] < 61:
        us_census2.loc[index, 'Cat_Current hours of work/week'] = 'Over time [41 - 61)'
        
        
    elif us_census2.loc[index, 'Current hours of work/week'] >= 61:
        us_census2.loc[index, 'Cat_Current hours of work/week'] = 'Over time [61 - inf)'
    
    
    # safety net
    else:
        us_census2.loc[index, 'Cat_Current hours of work/week'] = 'Hours not defined'


# variable Capital gain/loss
us_census2 ['Cat_Capital gain/loss'] = 0

# for loop with iterrows()
for index, col in us_census2.iterrows():
    
    
    # conditionals to change the values in the new column
    if us_census2.loc[index, 'Capital gain/loss'] < 0:
        us_census2.loc[index, 'Cat_Capital gain/loss'] = 'Negative [below 0 - 0)'
        
        
    elif us_census2.loc[index, 'Capital gain/loss'] < 5000:
        us_census2.loc[index, 'Cat_Capital gain/loss'] = '[0 - 5K)'
        
        
    elif us_census2.loc[index, 'Capital gain/loss'] < 10000:
        us_census2.loc[index, 'Cat_Capital gain/loss'] = '[5K - 10K)'
        
        
    elif us_census2.loc[index, 'Capital gain/loss'] >= 10000:
        us_census2.loc[index, 'Cat_Capital gain/loss'] = '[Over 10K]'
    
    
    # safety net
    else:
        us_census2.loc[index, 'Cat_Capital gain/loss'] = 'Capital gain/loss not defined'


# variable Completed years of education
us_census2 ['Cat_Completed years of education'] = 0

# for loop with iterrows()
for index, col in us_census2.iterrows():
    
    
    # conditionals to change the values in the new column, defined by education levels for the US
    if us_census2.loc[index, 'Completed years of education'] < 10:
        us_census2.loc[index, 'Cat_Completed years of education'] = 'Pre-school_K-12 [1 - 10)'
        
        
    elif us_census2.loc[index, 'Completed years of education'] == 10:
        us_census2.loc[index, 'Cat_Completed years of education'] = 'Some-college [10]'
        
        
    elif us_census2.loc[index, 'Completed years of education'] < 13:
        us_census2.loc[index, 'Cat_Completed years of education'] = 'Associate [11 - 13)'
        
        
    elif us_census2.loc[index, 'Completed years of education'] == 13:
        us_census2.loc[index, 'Cat_Completed years of education'] = 'Undergrad [13]'
        
        
    elif us_census2.loc[index, 'Completed years of education'] < 16:
        us_census2.loc[index, 'Cat_Completed years of education'] = 'Masters_prof school [14 - 16)'
        
                
    elif us_census2.loc[index, 'Completed years of education'] >= 16:
        us_census2.loc[index, 'Cat_Completed years of education'] = 'Doctorate [16 - inf)'
    
    
    # safety net
    else:
        us_census2.loc[index, 'Cat_Completed years of education'] = 'Completed years of education not defined'

<br>
<em><strong>Grouping categorical variables</strong></em>
<br>
Then we used a loop to group all of our categorical variables into predetermined bins, and create a new column where the resulting bins will be identified regarding the original data.
<br>
<em>-Job/occupation: </em>Grouped by the Standard Occupational Classification System.
<br>
<em>-Type of employment: </em>Grouped by private, public or (self or not employed).
<br>
<em>-Highest level of education: </em>Grouped by education levels for the US.
<br>
<em>-Martial status: </em>Grouped by currently married or not.
<br>
<em>-Relationship inside household: </em>Grouped by being a relative or not.
<br>
<em>-Ethnicity: </em>Grouped by black, white or other.
<br>
<em>-Country of birth: </em>Grouped by US vs others.
<br>

In [ ]:
# creating categories for categorical variables

# variable Job/occupation, defined by the Standard Occupational Classification System
us_census2['Cat_Job/occupation'] = us_census2['Job/occupation'].map(
                        {'Exec-managerial'     : 'Managerial',
                         'Prof-specialty'      : 'Professional',
                         'Craft-repair '       : 'Operational',
                         'Adm-clerical'        : 'Operational',
                         'Sales'               : 'Operational',
                         'Other-service'       : 'Operational',
                         'Machine-op-inspct'   : 'Operational',
                         'Transport-moving'    : 'Operational',
                         'Handlers-cleaners'   : 'Operational',
                         'Farming-fishing'     : 'Operational',
                         'Tech-support'        : 'Operational',
                         'Protective-serv'     : 'Operational',
                         'Priv-house-serv '    : 'Operational',
                         'Armed-Forces'        : 'Operational'})


# variable Type of employment, defined by private, public or (self or not employed)
us_census2['Cat_Type of employment'] = us_census2['Type of employment'].map(
                        {'Never-worked'     : 'Self or not employed',
                         'Without-pay'      : 'Self or not employed',
                         'Local-gov'        : 'Government-public',
                         'State-gov'        : 'Government-public',
                         'Federal-gov'      : 'Government-public',
                         'Private'          : 'Private',
                         'Self-emp-not-inc' : 'Self or not employed',
                         'Self-emp-inc'     : 'Self or not employed'})


# variable Highest level of education, defined by education levels for the US
us_census2['Cat_Highest level of education'] = us_census2['Highest level of education'].map(
                        {'Preschool'   : 'Pre-school_K-12',
                         '1st-4th'     : 'Pre-school_K-12',
                         '5th-6th'     : 'Pre-school_K-12',
                         '7th-8th'     : 'Pre-school_K-12',
                         '9th'         : 'Pre-school_K-12',
                         '10th'        : 'Pre-school_K-12',
                         '11th'        : 'Pre-school_K-12',
                         '12th'        : 'Pre-school_K-12',
                         'HS-grad'     : 'Pre-school_K-12',
                         'Some-college': 'Some-college',
                         'Assoc-voc'   : 'Associate',
                         'Assoc-acdm'  : 'Associate',
                         'Bachelors'   : 'Undergrad',
                         'Masters'     : 'Grad_masters_prof_sch', 
                         'Prof-school' : 'Grad_masters_prof_sch',
                         'Doctorate'   : 'Grad_doctorate'})

# variable Marital status, defined by currently married or not
us_census2['Cat_Marital status'] = us_census2['Marital status'].map(
                        {'Married-civ-spouse'    : 'Married',
                         'Married-AF-spouse'     : 'Married',
                         'Married-spouse-absent' : 'Married',
                         'Divorced'              : 'Not married',
                         'Separated'             : 'Not married',
                         'Widowed'               : 'Not married',
                         'Never-married'         : 'Not married'})

# variable Relationship inside the household, defined by being a relative or not
us_census2['Cat_Relationship inside the household'] = us_census2['Relationship inside the household'].map(
                        {'Not-in-family'  : 'Not_related',
                         'Husband'        : 'Related',
                         'Wife'           : 'Related',
                         'Own-child'      : 'Related',
                         'Unmarried'      : 'Related',
                         'Other-relative' : 'Related'})

# variable Ethnicity, defined by black, white or other
us_census2['Cat_Ethnicity'] = us_census2['Ethnicity'].map(
                        {'Black'              : 'Black',
                         'White'              : 'White',
                         'Asian-Pac-Islander' : 'Other ethnicity',
                         'Amer-Indian-Eskimo' : 'Other ethnicity',
                         'Other'              : 'Other ethnicity'})

# variable Country of birth, defined by US vs others.
for index, col in us_census2.iterrows():
    
    # conditionals to change the values in the new column, defined by  US or others
    if us_census2.loc[index, 'Country of birth'] == 'United-States':
        us_census2.loc[index, 'Cat_Country of birth'] = 'USA'
        
        
    elif us_census2.loc[index, 'Country of birth'] != 'United-States':
        us_census2.loc[index, 'Cat_Country of birth'] = 'Other'
        
           # safety net
    else:
        us_census2.loc[index, 'Cat_Country of birth'] = 'Country of birth not defined'

<br>
<em><strong>Printing the results for the grouping</strong></em>
<br>
Print statements were used with value counts for each new column containing the defined bins. This helped us identify that our division did not create stratas with too little observations and so the analysis could continue.
<br>

In [ ]:
# printing the value counts for each category
print(us_census2['Cat_Age'].value_counts())
print('-'*70)
print(us_census2['Cat_Job/occupation'].value_counts())
print('-'*70)
print(us_census2['Cat_Type of employment'].value_counts())
print('-'*70)
print(us_census2['Cat_Current hours of work/week'].value_counts())
print('-'*70)
print(us_census2['Cat_Capital gain/loss'].value_counts())
print('-'*70)
print(us_census2['Cat_Highest level of education'].value_counts())
print('-'*70)
print(us_census2['Cat_Completed years of education'].value_counts())
print('-'*70)
print(us_census2['Cat_Marital status'].value_counts())
print('-'*70)
print(us_census2['Cat_Relationship inside the household'].value_counts())
print('-'*70)
print(us_census2['Cat_Ethnicity'].value_counts())
print('-'*70)
print(us_census2['Gender'].value_counts())
print('-'*70)
print(us_census2['Cat_Country of birth'].value_counts())
print('-'*70)
print(us_census2['Income'].value_counts())
print('-'*70)

<br><br>
<strong>Getting dummy variables for the new categories</strong>
<br>
We created a DataFrame containing the dummy variables generated by using the get_dummies() function. This function was applied to the new categories created in the previous steps and the variables Gender and Income which didn't required any categorization because only two possible values already existed for these variables.
<br><br>
Additionally, since this new DataFrame duplicated the original columns for which we created the categories, we decided to drop them and generate a "cleaned" DataFrame containing no duplicate columns.
<br><br>

In [ ]:
# getting all dummies
us_census2_dummies = pd.get_dummies(data       = us_census2, 
                                    columns    = ['Cat_Age',
                                                  'Cat_Job/occupation',
                                                  'Cat_Type of employment',
                                                  'Cat_Current hours of work/week',
                                                  'Cat_Capital gain/loss',
                                                  'Cat_Highest level of education',
                                                  'Cat_Completed years of education',
                                                  'Cat_Marital status',
                                                  'Cat_Relationship inside the household',
                                                  'Cat_Ethnicity',
                                                  'Gender',
                                                  'Cat_Country of birth',
                                                  'Income'])

#dropping repeated columns
us_census2_cleaned = us_census2_dummies.drop(["Age",
                                      "Job/occupation",
                                      "Type of employment",
                                      "Current hours of work/week",
                                      "Capital gain/loss",
                                      "Highest level of education",
                                      "Completed years of education",
                                      "Marital status",
                                      "Relationship inside the household",
                                      "Ethnicity",
                                     # "Gender",
                                      "Country of birth", 
                                      #"Income",
                                      "m_Job/occupation",
                                      "m_Type of employment",
                                      "m_Country of birth"],
                                       axis = 1)

<br><br>
<strong>Concatenating the dummy variables with imputed data</strong>
<br>
Succeeding the generation of the dummy variables, we added them with the concatenate function to our imputed dataset.
<br>

In [ ]:
us_census3 = pd.concat([us_census, us_census2_cleaned], axis = 1)

<br><br>
<strong>Generating training and testing sets</strong>
<br>
Afterwards we split the data into training and testing sets for both our features and the dependent variable by taking 20% of the total records to be allocated as our testing set, this percentage was selected randomly. We did set a random seed so that we will always have the same selection if we decided to change our model. 
<br>
The X or features and Y or dependent variables training sets were concatenated into one single dataset, we did the same for X and Y testing sets.
<br><br>
Next, these two datasets were exported and saved as Excel files to be used in out Naive Bayes model.
<br>
It is important to clarify that we decided to drop every variable that was not a dummy variable from our data sets. This was done so that these wouldn't be exported as part of our Excel files because we won't be using these original columns in our analysis. 
<br><br>

In [ ]:
# preparing explanatory variable data, dropping columns and dependent variable
us_census3_data = us_census3.drop(['Age',
                                   'Job/occupation',
                                   'Type of employment',
                                   'Current hours of work/week',
                                   'Capital gain/loss',
                                   'Highest level of education',
                                   'Completed years of education',
                                   'Marital status',
                                   'Relationship inside the household',
                                   'Ethnicity',
                                   'Gender',
                                   'Country of birth',
                                   'Income',
                                   'm_Job/occupation',
                                   'm_Type of employment',
                                   'm_Country of birth',
                                   'Income_<=50K',
                                   'Income_>50K',
                                   'Cat_Age_[0 - 28)',
                                   'Cat_Job/occupation_Managerial',
                                   'Cat_Type of employment_Government-public',
                                   'Cat_Current hours of work/week_Full time [30 - 41)',
                                   'Cat_Capital gain/loss_Negative [below 0 - 0)',
                                   'Cat_Highest level of education_Associate',
                                   'Cat_Completed years of education_Associate [11 - 13)',
                                   'Cat_Marital status_Married',
                                   'Cat_Relationship inside the household_Not_related',
                                   'Cat_Ethnicity_Black',
                                   'Gender_Female',
                                   'Cat_Country of birth_Other'],
                                    axis = 1)

# preparing response variable data
us_census3_target = us_census3.loc[ : , ['Income_<=50K','Income_>50K']]

# preparing training and testing sets
x_train, x_test, y_train, y_test = train_test_split(
            us_census3_data,
            us_census3_target,
            test_size = 0.20,
            random_state = 1223)


# printing to validate the size of our sets
print(f"""
Training Data 
-------------
X-side: {x_train.shape}
y-side: {y_train.shape[0]}


Testing Data
------------
X-side: {x_test.shape}
y-side: {y_test.shape[0]}
""")


In [ ]:
# merging x_train and y_train so that they can be used in statsmodels
us_census3_train = pd.concat([x_train, y_train], axis = 1)

# merging x_test and y_test so that they can be used in statsmodels
us_census3_test = pd.concat([x_test, y_test], axis = 1)

In [ ]:
# # saving the training set as an Excel file
# us_census3_train.to_excel(excel_writer = "us_census_train.xlsx",
#                         index        = False)

# # saving the testing set as an Excel file
# us_census3_test.to_excel(excel_writer = "us_census_test.xlsx",
#                        index        = False)